<a href="https://colab.research.google.com/github/Pataweepr/applyML_vistec_2019/blob/master/hw7_Transfer_Learning(student).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning for image classification
In this lab, we are going to explore a few options to train models for image classification.
We will use [17 Category Flower Dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/17/) throughout this lab.

First, we will mount the dataset to this notebook.
Add this [zip file](https://drive.google.com/open?id=1UCRGAy6hin1Uhsscn150zVvGgKiaePmJ) to your google drive (by clicking google drive symbol on the top-right) and change the unzip path accordingly: 

The following command will mount your google drive to the local machine this notebook is running on. Authorize the mounting and then unzip the dataset. 

It should take a while to unzip.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
!unzip '/content/gdrive/My Drive/lab3-nvidia-chula.zip' > /dev/nul

## Import python libraries

We are downgrading Pillow. This will require you to **restart the runtime** after the following code block is done.

In [0]:
# Downgrade Pillow to avoid errors
!pip install Pillow==3.4.2

import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from time import time

from numpy.random import seed
seed(12)
from tensorflow import set_random_seed
set_random_seed(12)

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12)

from sklearn.metrics import classification_report, f1_score, accuracy_score
import glob
from tqdm import tqdm
import warnings

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras import backend as K
# This is called to clear the original model session in order to use TensorBoard
K.clear_session()

## Variable path Setting
- training, validation, testing and model path directories.

In [0]:
train_dir = "./dataset/lab3-1/10_flower/train"
val_dir = "./dataset/lab3-1/10_flower/validate"
test_dir = "./dataset/lab3-1/10_flower/test"

## Data preprocessing

Read training and validation dataset

### Data augmentation strategies using ImageDataGenerator

Keras has automatic [data augmentation](https://keras.io/preprocessing/image/) for images. Data augmentation helps increase the amount of training data and help reduce overfitting of the model. Image generator also helps manage the loading of data to reduce the amount of memory required. An example usage is shown below.


In [0]:
image_size = 224
num_class = 10

batch_size=32

train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
 
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(224,224),
        batch_size=batch_size,
        class_mode='categorical')

From the code above, what kind of data augmentation is being done?

**Ans: **

In [0]:
x, y = train_generator.next()
print(x.shape,y.shape)

Describe what each dimension of x and y is

**Ans:**

## Modeling

The base model (model architecture) we will used is called VGG19. The `VGG19` model are pre-trained on `ImageNet`. You can read more about VGG19 in [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

We can start with these pretrained weights when training on our new task by using it only as a feature extractor. That means that we freeze every layer prior to the output layer and simply learn a new output layer.

To fine-tune a network, we must first replace the last fully-connected layer with a new one that outputs the desired number of classes. We initialize its weights randomly. Then we continue training as normal. Sometimes it’s common to use a smaller learning rate based on the intuition that we may already be close to a good result.

In this demonstration, we’ll fine-tune a model pretrained on `VGG19` to the smaller target task and train the model with checkpoints and early stopping callbacks.

Define f1_score metric for evaluating model 

In [0]:
def f1_score_metric(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

## Model 1 (No pre-training)
- VGG19 (random initialized weights) + 2 Dense layers + Output layer

In [0]:
#K.clear_session()

In [0]:
base_model = VGG19(weights=None, include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=30, verbose=0, mode='max')

model_1_path = "./model/lab3_1/{}.h5".format("model_1")
checkpoint = ModelCheckpoint(model_1_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

# Create and compile model
model_1 = Model(inputs=base_model.input, outputs=output)
model_1.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001),metrics=["accuracy", f1_score_metric])

# Load last epoch model
# We have already trained the model for a certain amount of iterations.
# Otherwise, this will take too long.
pretrined_model_1_path = "./model/const_models/lab3_1/model_1.h5"
model_1.load_weights(pretrined_model_1_path)

## Run until early stopping
num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size
model_1.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=5,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps
        )


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
base_model = VGG19(weights=None, include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=30, verbose=0, mode='max')

model_1_path = "./model/lab3_1/{}.h5".format("model_1")
checkpoint = ModelCheckpoint(model_1_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

# Create and compile model
model_1 = Model(inputs=base_model.input, outputs=output)
model_1.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001),metrics=["accuracy", f1_score_metric])

# Load last epoch model
pretrined_model_1_path = "./model/const_models/lab3_1/model_1.h5"
model_1.load_weights(pretrined_model_1_path)

## Run until early stopping
num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size
model_1.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=5,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps
        )
        </code>
      </pre>
</details>

Notice how there are three callbacks in the code. Explain each of them

** Ans: **

## Model 2 (with frozen pre-trained weights)
- VGG (pre-trained weights) + 2 Dense layers + Output layer
- Freezes all weights of base model layers

In [0]:
#K.clear_session()

In [0]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=20, verbose=0, mode='max')

model_2_path = "./model/lab3_1/{}.h5".format("model_2")
checkpoint = ModelCheckpoint(model_2_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

# Create and compile model
model_2 = Model(inputs=base_model.input, outputs=output)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001),metrics=["accuracy", f1_score_metric])

# Load last epoch model
pretrined_model_2_path = "./model/const_models/lab3_1/model_2.h5"
model_2.load_weights(pretrined_model_2_path)

## Run until early stopping
num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size
model_2.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=5,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps
        )


<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=20, verbose=0, mode='max')

model_2_path = "./model/lab3_1/{}.h5".format("model_2")
checkpoint = ModelCheckpoint(model_2_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

# Create and compile model
model_2 = Model(inputs=base_model.input, outputs=output)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001),metrics=["accuracy", f1_score_metric])

# Load last epoch model
pretrined_model_2_path = "./model/const_models/lab3_1/model_2.h5"
model_2.load_weights(pretrined_model_2_path)

## Run until early stopping
num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size
model_2.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=5,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps
        )
        </code>
      </pre>
</details>

## Model 3 (with adaptable pre-trained weights)
- VGG19 (pre-trained weights) + 2 Dense layers + Output layer
- use basic Fine-tuning technique which unfreezes all layers.

In [0]:
#K.clear_session()

In [0]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=30, verbose=0, mode='max')

model_3_path = "./model/lab3_1/{}.h5".format("model_3")
checkpoint = ModelCheckpoint(model_3_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

# Create and compile model
model_3 = Model(inputs=base_model.input, outputs=output)
model_3.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.00001),metrics=["accuracy", f1_score_metric])

# Load last epoch model
pretrined_model_3_path = "./model/const_models/lab3_1/model_3.h5"
model_3.load_weights(pretrined_model_3_path)

## Run until early stopping
num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size
history = model_3.fit_generator(
                    train_generator,
                    steps_per_epoch=stepsPerEpoch,
                    epochs=5,
                    callbacks = callbacks_list,
                    validation_data = validation_generator,
                    validation_steps=validationSteps
                    )

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                                verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=30, verbose=0, mode='max')

model_3_path = "./model/lab3_1/{}.h5".format("model_3")
checkpoint = ModelCheckpoint(model_3_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

# Create and compile model
model_3 = Model(inputs=base_model.input, outputs=output)
model_3.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.00001),metrics=["accuracy", f1_score_metric])

# Load last epoch model
pretrined_model_3_path = "./model/const_models/lab3_1/model_3.h5"
model_3.load_weights(pretrined_model_3_path)

## Run until early stopping
num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size
history = model_3.fit_generator(
                    train_generator,
                    steps_per_epoch=stepsPerEpoch,
                    epochs=5,
                    callbacks = callbacks_list,
                    validation_data = validation_generator,
                    validation_steps=validationSteps
                    )
        </code>
      </pre>
</details>

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for f1_score
plt.plot(history.history['f1_score_metric'])
plt.plot(history.history['val_f1_score_metric'])
plt.title('model f1_score_metric')
plt.ylabel('f1_score_metric')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for f1_score
plt.plot(history.history['f1_score_metric'])
plt.plot(history.history['val_f1_score_metric'])
plt.title('model f1_score_metric')
plt.ylabel('f1_score_metric')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
        </code>
      </pre>
</details>

## Model 4 (with chain-taw)
- VGG (pre-trained weights) + 2 Dense layers + Output layer
- use `Chain-thaw` Fine-tuning technique, referenced by [Using millions of emoji occurrences to learn any-domain representations for detecting sentiment, emotion and sarcasm](https://arxiv.org/pdf/1708.00524.pdf).

In [0]:
#K.clear_session()

In [0]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Create  model
model_4 = Model(inputs=base_model.input, outputs=output)

num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size

base_model_layers = model_4.layers[:23]
new_model_layers =  model_4.layers[23:]

base_model_blocks = {
    0: base_model_layers[1:3],
    1: base_model_layers[4:6],
    2: base_model_layers[7:11],
    3: base_model_layers[12:16],
    4: base_model_layers[17:21]
}

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(num_class, activation='softmax')(x)

# Create  model
model_4 = Model(inputs=base_model.input, outputs=output)

num_training_img=240
num_validation_img=80
stepsPerEpoch = num_training_img/batch_size
validationSteps= num_validation_img/batch_size

base_model_layers = model_4.layers[:23]
new_model_layers =  model_4.layers[23:]

base_model_blocks = {
    0: base_model_layers[1:3],
    1: base_model_layers[4:6],
    2: base_model_layers[7:11],
    3: base_model_layers[12:16],
    4: base_model_layers[17:21]
}
        </code>
      </pre>
</details>

In [0]:
print("\n--[Phase 1]--: Train only new layers")

for layer in base_model_layers:
   layer.trainable = False

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                               verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                          patience=25, verbose=0, mode='max')

model_4_path = os.path.abspath(os.path.join("./model/const_models/lab3_1/{}.h5".format("model_4_1")))
checkpoint = ModelCheckpoint(model_4_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

model_4.compile(loss="categorical_crossentropy",
             optimizer=optimizers.Adam(lr=0.0001), metrics=["accuracy", f1_score_metric])

model_4.fit_generator(
   train_generator,
   steps_per_epoch=stepsPerEpoch,
   epochs=50,
   callbacks=callbacks_list,
   validation_data=validation_generator,
   validation_steps=validationSteps
   )

model_4_weight_path = "./model/lab3_1/{}.h5".format("model_4_1")
model_4.save_weights(model_4_weight_path)

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
print("\n--[Phase 1]--: Train only new layers")

for layer in base_model_layers:
   layer.trainable = False

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=10,
                               verbose=1, min_delta=0.0001, cooldown=5, min_lr=0)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                          patience=25, verbose=0, mode='max')

model_4_path = os.path.abspath(os.path.join("./model/const_models/lab3_1/{}.h5".format("model_4_1")))
checkpoint = ModelCheckpoint(model_4_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
callbacks_list = [checkpoint,early_stopper,reduce_lr]

model_4.compile(loss="categorical_crossentropy",
             optimizer=optimizers.Adam(lr=0.0001), metrics=["accuracy", f1_score_metric])

model_4.fit_generator(
   train_generator,
   steps_per_epoch=stepsPerEpoch,
   epochs=50,
   callbacks=callbacks_list,
   validation_data=validation_generator,
   validation_steps=validationSteps
   )

model_4_weight_path = "./model/lab3_1/{}.h5".format("model_4_1")
model_4.save_weights(model_4_weight_path)
        </code>
      </pre>
</details>

In [0]:
print("\n--[Phase 2]--: Train every CNN blocks of base model (5 blocks)")

model_4_prev_path = os.path.abspath(os.path.join("./model/const_models/lab3_1/model_4_1.h5"))
print("Loading the previous weight from {}.".format(model_4_prev_path))
model_4.load_weights(model_4_prev_path)

for layer in new_model_layers:
   layer.trainable = False

for idx in range(0, 5):
  # train idx block
  print("[Train block{}]: containing {} layers".format(idx, len(base_model_blocks[idx])))

  # Callbacks
  reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=4,
                                 verbose=1, min_delta=0.0001, cooldown=2, min_lr=0)
  early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                            patience=10, verbose=0, mode='max')

  model_4_path = os.path.abspath(os.path.join(f"./model/const_models/lab3_1/model_4_2_{idx}.h5"))
  checkpoint = ModelCheckpoint(model_4_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
  callbacks_list = [checkpoint,early_stopper,reduce_lr]

  for layer in base_model_layers:
    if layer in base_model_blocks[idx]:
      layer.trainable = True
      print("train {}".format(layer))
    else:
       layer.trainable = False
  model_4.compile(loss="categorical_crossentropy",
               optimizer=optimizers.Adam(lr=0.000001), metrics=["accuracy", f1_score_metric])
  model_4.fit_generator(
     train_generator,
     steps_per_epoch=stepsPerEpoch,
     epochs=50,
     callbacks=callbacks_list,
     validation_data=validation_generator,
     validation_steps=validationSteps
  )

<details>
    <summary>SOLUTION HERE!</summary>
      <pre>
        <code>
print("\n--[Phase 2]--: Train every CNN blocks of base model (5 blocks)")

model_4_prev_path = os.path.abspath(os.path.join("./model/const_models/lab3_1/model_4_1.h5"))
print("Loading the previous weight from {}.".format(model_4_prev_path))
model_4.load_weights(model_4_prev_path)

for layer in new_model_layers:
   layer.trainable = False

for idx in range(0, 5):
  # train idx block
  print("[Train block{}]: containing {} layers".format(idx, len(base_model_blocks[idx])))

  # Callbacks
  reduce_lr = ReduceLROnPlateau(monitor='val_acc', mode='max', factor=0.1, patience=4,
                                 verbose=1, min_delta=0.0001, cooldown=2, min_lr=0)
  early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, 
                            patience=10, verbose=0, mode='max')

  model_4_path = os.path.abspath(os.path.join(f"./model/const_models/lab3_1/model_4_2_{idx}.h5"))
  checkpoint = ModelCheckpoint(model_4_path, monitor='val_acc', verbose=1,save_best_only=True,save_weights_only=False, mode='max',period=1)
  callbacks_list = [checkpoint,early_stopper,reduce_lr]

  for layer in base_model_layers:
    if layer in base_model_blocks[idx]:
      layer.trainable = True
      print("train {}".format(layer))
    else:
       layer.trainable = False
  model_4.compile(loss="categorical_crossentropy",
               optimizer=optimizers.Adam(lr=0.000001), metrics=["accuracy", f1_score_metric])
  model_4.fit_generator(
     train_generator,
     steps_per_epoch=stepsPerEpoch,
     epochs=50,
     callbacks=callbacks_list,
     validation_data=validation_generator,
     validation_steps=validationSteps
  )
        </code>
      </pre>
</details>

## `To do` 

You are supposed to implement `Chain-thaw` Fine-tuning technique in the last phase which is to unfreeze all of model layers and train it to be well adapted to the target task.

HINT: you should load the last epoch model weights from provided const-model folder before training model with `fit_generator` function from the code below.


```python
pretrined_model_4_path = "./model/const_models/lab3_1/model_4_3.h5"
model_4.load_weights(pretrined_model_4_path)
```


Noted that there are still needs for both `ModelCheckpoint` callback, saving model to the path, and `EarlyStopping` is still required.

The solution can be seen in the cell below.

In [0]:
print("--[Phase 3]--: Train all layers")


## Evaluation
-  `F-score` is really useful if you want to compare 2 classifiers. It is computed using the harmonic mean of precision and recall, and gives much more weight to low values. As a result of that, the classifier will only get a high F-score, if both recall and precision are high. You can easily compute the F-Score with sklearn.
- `Micro F1-score` is to calculate metrics globally by counting the total true positives, false negatives and false positives, while 
- `Macro F1-score` is to calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

In [0]:
class_to_idx = train_generator.class_indices
idx_to_class = {v: k for k, v in class_to_idx.items()}

In [0]:
idx_to_class

In [0]:
!ls ./model/lab3_1/

In [0]:
# Load the best model
model_1_path = "./model/lab3_1/{}.h5".format("model_1")
model_2_path = "./model/lab3_1/{}.h5".format("model_2")
model_3_path = "./model/lab3_1/{}.h5".format("model_3")
model_4_path = "./model/lab3_1/{}.h5".format("model_4_1")

model_1.load_weights(model_1_path)
model_2.load_weights(model_2_path)
model_3.load_weights(model_3_path)
model_4.load_weights(model_4_path)

In [0]:
# This code just run the 4 models on the test set.

idx_to_model = {
    1: model_1,
    2: model_2,
    3: model_3,
    4: model_4
}
idx_to_preds = {
    1: ([], []), # y_trues, y_preds
    2: ([], []),
    3: ([], []),
    4: ([], [])
}

for class_idx in tqdm(range(num_class)):
    label = idx_to_class[class_idx]
    file_list = glob.glob("{}/{}/*.jpg".format(test_dir, label))
    for raw_image in file_list:
        inputShape = (224,224) # Assumes 3 channel image
        image = load_img(raw_image, target_size=inputShape)
        image = img_to_array(image)   # shape is (224,224,3)
        image = preprocess_input(image)
        image = np.expand_dims(image, axis=0)  # Now shape is (1,224,224,3)

        for model_idx in range(1,5):
            model = idx_to_model[model_idx]
            preds = model.predict(image)
            pred_class = np.argmax(preds)

            # append y_trues
            idx_to_preds[model_idx][0].append(class_idx)
            # append y_preds
            idx_to_preds[model_idx][1].append(pred_class)

In [0]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print("Performace of each models:")
    for idx in range(1, 5):
      
        y_trues = idx_to_preds[idx][0]
        y_preds = idx_to_preds[idx][1]
        print("-- Model_{} (acc:{:.4f}, f1_micro:{:.4f}, f1_macro:{:.4f})".format(idx,
                                                                                accuracy_score(y_trues, y_preds), 
                                                                                f1_score(y_trues, y_preds, average='micro'),
                                                                                f1_score(y_trues, y_preds, average='macro')))

From the results, what gives the biggest gain in accuracy?

** Ans: **

## Result
To see how the model performs for the testing samples, we can visualize them using matplotlib.

In [0]:
import random

for class_idx in random.sample(range(0, num_class), 3):
    label = idx_to_class[class_idx]
    file_list = glob.glob("{}/{}/*.jpg".format(test_dir, label))
    for raw_image in file_list[:2]:
        inputShape = (224,224)
        image = load_img(raw_image, target_size=inputShape)
        image = img_to_array(image)
        image = preprocess_input(image)
        image = np.expand_dims(image, axis=0)

        preds = model_4.predict(image)

        pred_class = np.argmax(preds)
        pred_label = idx_to_class[pred_class]

        title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
            label,
            pred_label,
            preds[0][pred_class])

        original = load_img(raw_image, target_size=(224, 224))
        plt.figure(figsize=[3,3])
        plt.axis('off')
        plt.title(title)
        plt.imshow(original)
        plt.show()